In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Vertex AI Vector Search for StackOverflow Questions

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fvector_search%2Fsdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This example demonstrates how to encode custom text embeddings using the StackOverflow dataset and the sentence-T5 model. These are uploaded to the Vertex AI Vector Search service. It's a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it's a fully managed offering, further reducing operational overhead. The Vertex AI Vector Search service is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

**Pre-requisite**: This notebook requires you to already have a VPC network set up. See the "Prepare a VPC network" section in [Create Vertex AI Vector Search index notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb).

Learn more about [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview).

### Objective

In this notebook, you learn how to encode custom text embeddings, create an Approximate Nearest Neighbor (ANN) index, and query against indexes.

This tutorial uses the following Google Cloud ML services:

- Vertex AI Vector Search

The steps performed include:

* Create ANN index.
* Create an index endpoint with VPC Network.
* Deploy ANN index.
* Perform online query.


### Dataset

The dataset used for this tutorial is the [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow).

> Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
# Install the google-cloud packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage \
                         'google-cloud-bigquery[pandas]' -q

# Install the latest version of tensorflow packages
! pip3 install --upgrade tensorflow \
                         tensorflow_text \
                         tensorflow-hub -q

# Install the redis and tqdm packages
! pip install --upgrade redis \
                        tqdm -q

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

Learn more about how to [set up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [ ]:
import json
import os
import tempfile
from typing import List

import numpy as np
import redis
import tensorflow as tf
import tensorflow_hub as hub
# Registers the ops.
import tensorflow_text as text  # noqa: F401
from google.cloud import bigquery
from tqdm.auto import tqdm

## Prepare the data

For this tutorial, use the [Stack Overflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) of question and answers hosted on BigQuery.

> This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. Each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

Fetch the dataset from the BigQuery source.

In [ ]:
%%time
client = bigquery.Client(project=PROJECT_ID)

NUM_ROWS = 1000

QUERY = f"""
        SELECT distinct q.id, q.title, q.body, q.tags, a.body as answers, a.score 
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
        INNER JOIN (SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers`  where Score>0 ORDER BY Score desc) AS a ON q.id = a.parent_id 
        where q.tags like '%python%'
        LIMIT {NUM_ROWS};
        """

query_job = client.query(QUERY)
rows = query_job.result()

In [ ]:
# Convert to a dataframe
df = rows.to_dataframe()

# Examine the data
df.head()

In [ ]:
# Extract the question ids and question text
ids = df.id.tolist()
questions = df.title.tolist()

# Verify the length
len(ids)

### Instantiate the text encoding model

Use the [sentence-t5 encoder](https://tfhub.dev/google/sentence-t5/st5-base/1) developed by Google for converting text to embeddings.

> The sentence-T5 family of models encode text into high-dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language processing tasks.
>
> The model is built on top of T5 (i.e., the Text-To-Text Transfer Transformer). It's trained on a variety of data sources and initialized from pre-trained T5 models with different model sizes. The input is variable-length English text and the output is a 768-dimensional vector. The sentence-T5 base model employs a 12-layer transformer architecture as does the T5 base model.

In [ ]:
hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"

encoder = hub.KerasLayer(hub_url)

### Define an encoding function

Define a function, to be used later, that takes sentences and converts them to embeddings.

In [ ]:
def encode_text_to_embedding(
    text_encoder: hub.KerasLayer, sentences: List[str], batch_size: int = 100
) -> np.ndarray:
    embeddings_list = []

    # Process data in chunks to prevent out-of-memory errors
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[i : i + batch_size]
        embeddings_list.append(text_encoder(tf.constant(batch)))

    return np.squeeze(np.column_stack(embeddings_list))

#### Test the encoding function

Encode a subset of data and see if the embeddings and distance metrics make sense.

According to the [sentence-T5 research paper](https://arxiv.org/pdf/2108.08877.pdf), the similarity of embeddings is calculated using the dot-product. 

In [ ]:
# Encode 500 questions
questions = df.title.tolist()[:500]
question_embeddings = encode_text_to_embedding(
    text_encoder=encoder, sentences=questions
)

Save the dimension size for later usage when creating the index.

In [ ]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

In [ ]:
question_index = 0

# Print the query question
print(f"Query question = {questions[question_index]}")
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

### Save the train split in JSONL format.

The data must be formatted in JSONL format, which means each embedding dictionary is written as a JSON string on its own line.

See more information in the docs for [input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure#data-file-formats).

In [ ]:
# Create temporary file to write embeddings to
embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

print(embeddings_file.name)

In [ ]:
# Set batch size
BATCH_SIZE = 100

# Create embeddings and write to a file
with open(embeddings_file.name, "a") as f:
    for i in tqdm(range(0, len(questions), BATCH_SIZE)):
        id_chunk = ids[i : i + BATCH_SIZE]

        question_chunk_embeddings = encode_text_to_embedding(
            text_encoder=encoder, sentences=questions[i : i + BATCH_SIZE]
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk, question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

Upload the training data to a Cloud Storage bucket.

In [ ]:
UNIQUE_FOLDER_NAME = "embeddings_folder_unique"
remote_folder = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {embeddings_file.name} {remote_folder}

## Create Indexes


### Create ANN Index (for Production Usage)

In [ ]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "questions from stackoverflow"

Create the ANN index configuration:

To learn more about configuring the index, see [Vector Search input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup).


In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

Using the resource name, you can retrieve an existing MatchingEngineIndex resource.

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## Setup VPC peering network

To use a Vector Search index, set up a VPC peering network between your project and the Vertex AI Vector Search service project. This eliminates additional hops in network traffic and allows using efficient gRPC protocol.

Learn more about [VPC peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering).

**IMPORTANT: you can only setup one VPC peering to servicenetworking.googleapis.com per project.**

### Create VPC peering

For simplicity, set up VPC peering to the `ucaip-haystack-vpc-network` network. You can create a different network for your project.

If you set up VPC peering with any other network, make sure that the network already exists and that your VM is running on that network.

In [ ]:
# This is for display only; you can name the range anything.
NETWORK = "ucaip-haystack-vpc-network"  # @param {type:"string"}
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 is
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### Create the VPC connection

Next, create the connection for VPC peering.

**Note:** If you get a PERMISSION DENIED, you may not have the neccessary 'Compute Network Admin' role set for your default service account. In the Cloud Console, do the following:

1. Goto **IAM & Admin**.
2. Find your service account.
3. Click edit icon.
4. Select **Add Another Role**.
5. Enter **Compute Network Admin**.
6. Select **Save**.

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

Check the status of your peering connections.

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

#### Construct the full network name

You need to have the full network resource name when you subsequently create an Vector Search index endpoint resource for VPC peering.

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

full_network_name = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"
print(full_network_name)

## Create an IndexEndpoint with VPC Network

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    network=full_network_name,
)

## Deploy Indexes

### Deploy ANN Index

In [ ]:
# Set a unique id for your deployed index
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

In [ ]:
# Deploy your ANN index to the index endpoint
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

## Create Online Queries

After you've built your indexes, you can query against the deployed index to find nearest neighbors.

**Note:** For the **DOT_PRODUCT_DISTANCE** distance type, the "distance" property returned with each MatchNeighbor actually refers to the similarity.

In [ ]:
test_embeddings = encode_text_to_embedding(
    text_encoder=encoder, sentences=["How do I install tensorflow with GPU support?"]
)

In [ ]:
# Test query
NUM_NEIGHBOURS = 20

response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=[test_embeddings.tolist()],
    num_neighbors=NUM_NEIGHBOURS,
)

response

Print titles to verify neighbors make sense

In [ ]:
neighbor_ids = [neighbor.id for neighbor in response[0]]
neighbor_distances = [neighbor.distance for neighbor in response[0]]

for match_index, neighbor in enumerate(response[0]):
    titles = df[df.id.astype(str) == neighbor.id].title.tolist()

    if len(titles) > 0:
        print(
            f"{match_index}: title = '{titles[0]}', distance = {neighbor.distance:0.2f}"
        )

## Storing and retrieving titles from a Redis data store
When you productionize this code into a service, you need to convert the nearest ids returned from Vertex AI Vector Search into usable data for downstream services.

In this case, you need to convert the ids to titles.

You can use Google Cloud's Memorystore to deploy a managed Redis instance to save the id-title key-value pairs.

See more information on [Memorystore](https://cloud.google.com/memorystore/docs/redis/create-manage-instances?hl=en).

In [ ]:
# Set a display name for your Redis instance
REDIS_INSTANCE_NAME = "stackoverflow-questions-unique"

# Create a Redis instance
! gcloud redis instances create '{REDIS_INSTANCE_NAME}' --size=5 --region={LOCATION} --network={VPC_NETWORK_FULL} --connect-mode=private-service-access

In [ ]:
# Get host and port info
if not os.getenv("IS_TESTING"):
    REDIS_HOST = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {LOCATION}  --format='value(HOST)'
    REDIS_PORT = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {LOCATION} --format='value(PORT)'

    if isinstance(REDIS_HOST, list):
        REDIS_HOST = REDIS_HOST[0]

    if isinstance(REDIS_PORT, list):
        REDIS_PORT = REDIS_PORT[0]

    print(f"REDIS_HOST = {REDIS_HOST}")
    print(f"REDIS_PORT = {REDIS_PORT}")

In [ ]:
# Connect to the instance
if not os.getenv("IS_TESTING"):
    redis_client = redis.StrictRedis(host=REDIS_HOST, port=REDIS_PORT)

In [ ]:
# Convert the id -> title relationship into a dict and write to redis
if not os.getenv("IS_TESTING"):
    redis_client.mset({str(id): str(title) for id, title in zip(df.id, df.title)})

In [ ]:
# Verify that redis can retrieve the correct information
if not os.getenv("IS_TESTING"):
    [
        f"Actual = {title}, Retrieved = {redis_client.get(str(id))}"
        for id, title in list(zip(df.id, df.title))[:10]
    ]

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()

# Delete cloud storage bucket
delete_bucket = False
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Delete redis instance
! gcloud redis instances delete '{REDIS_INSTANCE_NAME}' --region {LOCATION} --quiet